In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
import matplotlib.colors as mcolors
import torch
from learning_dynamics.data_modules import PendulumReconDataModule, PendulumExtrapDataModule
from learning_dynamics.models import PendulumRBFVAEModel, Encoder, Decoder
from learning_dynamics.callbacks import PendulumPEGPVAEPlotting
import wandb
from lightning.pytorch.callbacks import ModelCheckpoint
import lightning as L
torch.set_default_dtype(torch.float64)

In [3]:
train_data_path = '../../data/ODEs/pendulum/nonlinear.pkl'

In [4]:
batch_size = 32
data_module = PendulumReconDataModule(train_data_path=train_data_path, batch_size=batch_size, val_fraction=0.1, test_fraction=0.1)
data_module.setup("fit")

In [5]:
# params
width = data_module.train_dataset.dataset.width
height = data_module.train_dataset.dataset.height
embed_dim = 128
latent_dim = 1
dt = data_module.train_dataset.dataset.dt
val_check = 100
freq = 3.13
damp = 0.07
ls = 1
file = f"exp_a_exp_s_video_physics"
max_epochs = 6000

In [6]:
checkpoint_callback = ModelCheckpoint(
    dirpath=f"model_checkpoints/{file}",          
    filename="model-{epoch:03d}",  
    save_top_k=-1,                 
    every_n_epochs=val_check,               
    save_on_train_epoch_end=False,
    save_last=True
)

In [7]:
# ckpt_path = f"model_checkpoints/{file}/model-epoch=3999.ckpt" 
# trainer = L.Trainer(devices=[1], max_epochs=max_epochs, callbacks=[PendulumPEGPVAEPlotting(dt=dt, file=file, vids_per_batch=0), checkpoint_callback], check_val_every_n_epoch=val_check, logger=False)
# trainer.fit(model, datamodule=data_module, ckpt_path="last")

In [11]:
ckpt_path = f"model_checkpoints/{file}/model-epoch=649.ckpt" 

encoder = Encoder(width, height, embed_dim, latent_dim)
decoder = Decoder(width, height, embed_dim, latent_dim)

trainer = L.Trainer(devices=[1], max_epochs=max_epochs, callbacks=[PendulumPEGPVAEPlotting(dt=dt, file=file, vids_per_batch=1), checkpoint_callback], check_val_every_n_epoch=val_check, logger=False)
model = PendulumRBFVAEModel(encoder, decoder, dt, length_scale=ls, trainable_ls=False, amp=False, scale=True)
trainer.fit(model, datamodule=data_module)

# model = FingerPEGPVAEModel.load_from_checkpoint(ckpt_path, encoder=encoder,decoder=decoder, dt=dt, dq=dq, cq=cq, ls=ls, kld_max=kld_max, kld_schedule=kld_schedule)
# trainer.fit(model, datamodule=data_module, ckpt_path="last")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/kendra/miniconda3/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/kendra/Code/Learning-Dynamics-Latent-Force/notebooks/Pendulum/model_checkpoints/exp_a_exp_s_video_physics exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type    | Params | Mode 
-------------------------------------------------
0 | encoder      | Encoder | 409 K  | train
1 | decoder      | Decoder | 413 K  | train
  | other params | n/a     | 

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/home/kendra/miniconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kendra/Code/Learning-Dynamics-Latent-Force/src/learning_dynamics/gaussian_process/kernels.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  length_scale = torch.tensor(length_scale, dtype=x1.dtype, device=x1.device)
/home/kendra/Code/Learning-Dynamics-Latent-Force/src/learning_dynamics/callbacks.py:156: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/home/kendra/miniconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connecto

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...

KeyboardInterrupt

